In [30]:
import pandas as pd
import os
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


# Loads labels
df = pd.read_csv('../data/seedling_labels_with_features_and_predictions.csv')

#df = df.drop(columns=['Pos','average_expert'])
#df = df.iloc[:, -8:]
decisions = df.loc[:,['Expert 1','Expert 2','Expert 3', 'Expert 4']]
df

FileNotFoundError: [Errno 2] No such file or directory: '../data/seedling_labels_with_features_and_predictions.csv'

In [ ]:
# Define a function to convert the values to binary
def convert_to_binary(x):
    if x in [1, 2]:
        return 1
    else:
        return 0

# Apply the function to each element in the dataframe
decisions = decisions.applymap(convert_to_binary)
features = df.iloc[:,10:16]
voting = pd.concat([features,decisions],axis = 1)
voting.head()

In [ ]:
##MAJORITY VOTING

# define ground truth as the expert majority vote with 75%+ agreement
true_labels = df["expert_binary"]


In [31]:
from scipy.stats import beta
# Define the fusion function
def bayesian_fusion(expert_decisions):
    # Compute the beta distribution parameters based on expert decisions
    alpha = expert_decisions.sum()
    beta_ = expert_decisions.size - alpha
    
    # Compute the posterior probability distribution
    p = beta.cdf(0.5, alpha, beta_)
    
    return p

# Apply the fusion function to the dataset
voting['bayes_consensus'] = decisions.apply(bayesian_fusion, axis=1)

# Define the decision threshold
threshold = 0.5

# Make final classification decisions based on the threshold
voting['bayes_consensus'] = (voting['bayes_consensus'] > threshold).astype(int)
print(voting.head())

   plant_width_side_view  plant_height_side_view  plant_area_side_view  \
0                    120                     239                  8577   
1                    591                     407                 29037   
2                    388                     449                 31946   
3                     92                     242                  8694   
4                    582                     494                 42052   

   plant_width_top_view  plant_height_top_view  plant_area_top_view  Expert 1  \
0                   263                    277                18368         0   
1                   585                    548                85112         1   
2                   450                    692                86757         1   
3                   281                    276                23174         0   
4                   670                    534               102178         0   

   Expert 2  Expert 3  Expert 4  bayes_consensus  
0         0      

In [32]:
import numpy as np
from sklearn.model_selection import train_test_split
import pickle

from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from scipy.stats import mode
from scipy.stats import bayes_mvs

X = voting.iloc[:, 0:7]
y = true_labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=57)


In [33]:
# Train different classifiers
clf1 = RandomForestClassifier(random_state=57)
clf2 = AdaBoostClassifier(random_state=57)
clf3 = LogisticRegression(random_state=57)
clf4 = GaussianNB()

clf1.fit(X_train, y_train)
clf2.fit(X_train, y_train)
clf3.fit(X_train, y_train)
clf4.fit(X_train, y_train)


with open('clf1.pkl', 'wb') as file:
    pickle.dump(clf1,file)

with open('clf2.pkl', 'wb') as file:   
    pickle.dump(clf2,file)

with open('clf3.pkl', 'wb') as file:
        pickle.dump(clf3,file)
    
with open('clf4.pkl', 'wb') as file:
        pickle.dump(clf4,file)



In [42]:
y_pred1 = clf1.predict(X_test)
y_pred2 = clf2.predict(X_test)
y_pred3 = clf3.predict(X_test)
y_pred4 = clf4.predict(X_test)

y_preds = np.vstack((y_pred1, y_pred2, y_pred3, y_pred4))
y_preds_to_save = np.transpose(y_preds)
y_test_to_save = np.array(y_test)
np.savetxt("models_predictions.csv", y_preds_to_save, delimiter=",")
np.savetxt("true_answer.csv", y_test_to_save, delimiter=",")


In [43]:
from sklearn.metrics import confusion_matrix
conf1 = confusion_matrix(y_test, y_pred1, normalize = 'true')
conf2 = confusion_matrix(y_test, y_pred2, normalize = 'true')
conf3 = confusion_matrix(y_test, y_pred3, normalize = 'true')
conf4 = confusion_matrix(y_test, y_pred4, normalize = 'true')

print(conf1)
print(conf2)
print(conf3)
print(conf4)

[[0.88059701 0.11940299]
 [0.03787879 0.96212121]]
[[0.91044776 0.08955224]
 [0.06060606 0.93939394]]
[[0.8358209  0.1641791 ]
 [0.06060606 0.93939394]]
[[0.7761194  0.2238806 ]
 [0.08333333 0.91666667]]


In [37]:
from scipy.stats import bayes_mvs

def decision_fusion(y_preds, method='hard'):
    if method == 'hard':
        return mode(y_preds, axis=0)[0][0]
    elif method == 'soft':
        return np.round(np.mean(y_preds, axis=0))
    elif method == 'bayesian':
        return 0
    else:
        raise ValueError("Invalid method")


In [38]:
y_pred_hard = decision_fusion(y_preds, method='hard')
y_pred_soft = decision_fusion(y_preds, method='soft')
y_pred_bayesian = decision_fusion(y_preds, method='bayesian')


In [39]:
print("Hard Voting Accuracy:", accuracy_score(y_test, y_pred_hard))
print("Soft Voting Accuracy:", accuracy_score(y_test, y_pred_soft))
#print("Bayesian Consensus Accuracy:", accuracy_score(y_test, y_pred_bayesian))


ValueError: Found input variables with inconsistent numbers of samples: [199, 4]